# 情報科学演習II アルゴリズムとデータ構造II 第2週 3冊目

#### 第2週目次

1. ダイクストラのアルゴリズムの簡単な実装（1冊目）
2. 実装のクラス化（2冊目）
3. 優先度付きキュー（3冊目）
4. ダイクストラのアルゴリズムの効率的な実装（4冊目）

## 3. 優先度付きキュー

ダイクストラのアルゴリズムでmin優先度付きキューを使うと$Q$から最短距離の頂点を効率的に取り出せるようになります．
次章で作成する`SecondImplementaion`クラスでリスト型`Q`の代わりにこのキューを使います．

この章ではmin優先度付きキューの実装例として`DijkstraHeap`クラスを示します．
データ構造にヒープを使うので，クラス名をヒープにしました．
この実装を次章で使えるように，min優先度付きキューのそれぞれの関数の使い方を理解してください．

次の2つのセルは準備ですので，実行してください．
二つ目のセルで必要なモジュールを`import`し，いくつかの関数を定義しておきます．

In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz
# 最終行は Successfully installed pygraphviz-x.xx
# x.xxの部分は1.13などのバージョン番号

In [ ]:
import networkx as nx
import math
import pygraphviz as pgv
from IPython.display import SVG, display_svg # Imageクラス
from test import support # sortdict関数

log = False # log = Trueにすると途中経過が出力される

### 3.1 実装例

ヒープクラスのデータは次の3つです．これらは`__init__`関数内で宣言しています．
- `d`：`d[u]`が頂点`u`の距離．Pythonの辞書型
- `Q`：距離順に頂点を格納したヒープ．`Q[0]`に最短距離の頂点．Pythonのリスト型
- `pos`：`pos[u]`は頂点`u`のヒープ`Q`の位置(添字)．`Q[pos[u]]`に頂点`u`が格納されている．Pythonの辞書型

ヒープクラスの関数は次の5つです．
- `extract_min()`：ヒープから最短距離の頂点を取り出す
- `insert(u)`：ヒープに頂点`u`を追加する
- `heapify_up(i)`：ヒープ条件を満たすように，ヒープの`i`番目の頂点を上に移動する
- `heapify_down(i)`：ヒープ条件を満たすように，ヒープの`i`番目の頂点を下に移動する
- `draw()`：ヒープを図示する．ヒープの動作を確認するための関数

`heapify_up`と`heapify_down`の二つの関数の引数はヒープの位置を与えることに注意せよ．頂点が引数なのは`insert`のみである．

In [ ]:
class DijkstraHeap:
  # ヒープは2分木．
  # Q[0]が木の根
  # i>=0のときQ[i]の左の子はQ[2*i+1]，右の子はQ[2*i+2]
  # i>0のときQ[i]の親はQ[(i-1)//2]
  # ヒープ条件：ヒープ内で子頂点の距離は親頂点の距離より長く，距離が最短の頂点がヒープの先頭にある．

  def __init__(self):
    self.d = {}   # d[u]は頂点uの距離
    self.Q = []   # 頂点が距離順に格納されたヒープ．Q[0]が最小距離の頂点
    self.pos = {} # pos[u]は頂点uのヒープQの位置(添字)．Q[pos[u]]に頂点uが格納されている．

  def extract_min(self): # ヒープから最短距離の頂点を取り出す
    Q = self.Q
    pos = self.pos
    if len(Q) < 1: # ヒープに頂点がない場合はエラー
      raise Exception("ヒープアンダーフロー")
    # ヒープの先頭にある頂点が最短距離の頂点
    min = Q[0]     
    pos[min] = -1  
    # ヒープの末尾にある頂点をヒープの先頭Q[0]に移動
    Q[0] = Q[len(Q) - 1]
    pos[Q[0]] = 0
    del Q[len(Q) - 1]
    # ヒープ条件を満たすようにQ[0]を移動する
    if len(Q) > 1:
      self.heapify_down(0)
    # 最短距離の頂点を返す
    return min

  def insert(self, u): # ヒープに頂点uを追加する
    # ヒープ条件を維持しないことに注意．
    # ヒープ条件を維持するにはinsertの後にheapify_up(Q, pos[u])を呼び出すこと．
    Q = self.Q
    pos = self.pos
    # 頂点uをヒープの末尾に追加．すでにuがヒープに存在する場合は追加しない
    if u not in pos:
      Q.append(u)         
      pos[u] = len(Q) - 1

  def heapify_up(self, i): # ヒープ条件を満たすように，ヒープのi番目の頂点を上に移動する
    d = self.d
    Q = self.Q
    pos = self.pos
    # ヒープのi番目の頂点をssとし，ssの距離をsdとする
    ss = Q[i]
    sd = d[ss]

    # 頂点ssをssの祖先と比べて，ヒープ条件を満たす位置を探す
    pi = (i - 1) // 2 # 親の添字．Q[i]の親がQ[pi]
    while pi > -1: # 親が存在する間，
      # 頂点ssの祖先の頂点をpとし，pの距離をpdとする
      p = Q[pi]
      pd = d[p]
      # 祖先pの距離が頂点ssの距離より長くなければヒープ条件を満たすのでwhileループを終了する
      if pd <= sd:
        break
      # 親pの距離が頂点ssの距離より長いので，親pを子の位置Q[i]に移動する
      if log:
        print('heapify_up 頂点ss', ss, '祖先', p, 'ssの距離', sd, '祖先の距離', pd)
      Q[i] = p
      pos[p] = i
      # 親のさらに親についてwhileの次の繰り返しでヒープ条件を満たすように処理する
      i = pi
      pi = (i - 1) // 2
    
    # ヒープのi番目の位置に頂点ssを格納する．iはwhileループで変わる場合があるので注意すること．
    Q[i] = ss
    pos[ss] = i

  def heapify_down(self, i): # ヒープ条件を満たすように，ヒープのi番目の頂点を下に移動する
    d = self.d
    Q = self.Q
    pos = self.pos
    # ヒープに格納されている頂点数がQlen
    Qlen = len(Q)
    # ヒープのi番目の頂点をppとし，ppの距離をpdとする
    pp = Q[i]
    pd = d[pp]

    # 頂点ppとppの子孫を比べて，ヒープ条件を満たす位置に頂点pを移動する．
    li = 2 * i + 1 # ヒープのi番目の左の子の添字
    while li < Qlen:
      # 頂点ppの左右の子のうち距離が短い方の頂点をsとし，sのヒープ上の位置をsi，sの距離をsdとする．
      # ひとまず左の子をsとする．
      si = li
      s = Q[si]
      sd = d[s]
      # 頂点ppに右の子が存在し，左の子より距離が短いならば，右の子をsとする．
      ri = li + 1 # 右の子の添字
      if ri < Qlen: # 右の子が存在する
        r = Q[ri]
        rd = d[r]
        if rd < sd: # 左の子より右の子の方が距離が短い．
          si = ri
          s = r
          sd = rd
      # 親pの距離が子sの距離より長くなければヒープ条件を満たすのでwhileループを終了する
      if pd <= sd:
        break
      # 親pの距離が子sの距離より長いので，親pを子の位置Q[i]に移動する
      if log:
        print('heapify_down 頂点pp', pp, '子孫', s, 'ppの距離', pd, '子孫の距離', sd)
      Q[i] = s
      pos[s] = i
      # 子のさらに子についてwhileの次の繰り返しでヒープ条件を満たすように処理する
      i = si
      li = 2 * i + 1 # ヒープのi番目の左の子の添字

    # 親のところに先頭の頂点を移動する
    Q[i] = pp
    pos[pp] = i

  def draw(self, filename = 'tmp'): # ヒープを図示する．ヒープの動作を確認するための関数
    d = self.d
    Q = self.Q
    A = pgv.AGraph(directed=True)
    for i in range(len(Q)):
      u = Q[i]
      if u in d:
        # print(i, Q[i], d[Q[i]])
        ud = d[u]
        A.add_node(i, label=f"<{u}<br/><font point-size=\"10pt\">d[{u}]={ud}</font>>")
      else:
        # print(i, Q[i])
        A.add_node(i, label=f"{u}")
      if i > 0:
        A.add_edge((i-1)//2, i)
    # ダミーの挿入
    if len(Q) % 2 == 0:
      A.add_node(len(Q), label='d', color='white', fontcolor='white')
      A.add_edge((len(Q)-1)//2, len(Q), color='white')
    A.layout('dot', args='-Nshape=box') # circle, oval, ellipse
    A.draw(F'{filename}.svg')
    display_svg(SVG(F'{filename}.svg'))


次の図の例を使ってヒープを説明する．

![](figs/heap.svg)

ヒープは二分木の形をしており，子の頂点の距離は親の頂点の距離以上（長いか等しい）であるという**ヒープ条件**を満たすように維持される．
図の頂点7の親に注目すると，その親は頂点0．
頂点7の距離は8，7の親の距離は0．
8は0以上なので頂点7と0はヒープ条件を満たしている．
頂点7の子に注目すると，その子は頂点4と2である．
頂点7の子頂点の距離は9と17であり，どちらも8以上の距離である．
よってこの親子もヒープ条件を満たしている．

ヒープ条件を満たすことから，根には最短距離の頂点がある．

ヒープに格納された頂点は，次の表のようにリストQの先頭から順に隙間なく保存される．
リストに二分木を格納するため，親子関係として，Q[i]の子をQ[2\*i+1]とQ[2\*i+2]と定める．
この例ではQ[1]に頂点7が格納されており，頂点7の二分木における子はQ[3]とQ[4]に格納された頂点4と2である．

| リストQの位置i | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
|--|--|--|--|--|--|--|--|--|
| 頂点u = Q[i] | 0 | 7 | 1 | 4 |  2 |  5 |  6 |  3 |
| 距離d[u]     | 0 | 8 | 5 | 9 | 17 | 14 | 29 | 20 |

次のセルでこのヒープを作る`demo()`関数を定義しているので実行すること．
この関数は`DijkstraHeap`オブジェクトを生成して返す．

In [ ]:
def demo():
  h = DijkstraHeap()
  for u, ud in zip([0,2,1,3,4,5,6,7], [0,17,5,20,9,14,29,8]):
    h.d[u] = ud
    print('追加する頂点', u, '距離', ud)
    h.insert(u)
    h.heapify_up(h.pos[u])
    if log:
      h.draw()
  return h

### 3.2 ヒープを図示する
`DijkstraHeap`クラスにヒープを図示する`draw()`関数がある．
この関数はヒープの動きを理解するための関数である．
この関数は効率的なアルゴリズムにするためには寄与していない．

**練習**
`draw()`関数の呼び出しを追加して，ヒープhを図示するコードを次のセルの`....`の部分に書きなさい．
上の図と同じ二分木が表示されたら正解．

ヒント：`h`は`DijkstraHeap`オブジェクトなので，`h.draw()`で`draw()`関数が呼び出せる．

In [ ]:
log = False # log = Trueにすると途中経過が出力される
h = demo()
# ヒープを図示するコードを書き加える
....

### 3.3 ヒープから最短距離の頂点を取り出す
ヒープ`h`から最短距離の頂点を取り出す関数は`extract_min`関数です．
`extract_min`関数は，引数のない関数で，頂点を返します．
`extract_min`関数内でヒープ条件を満たすために`heapify_down`関数を使っています．
ヒープ`h`の外からこの関数を呼び出すには，`h.extract_min()`関数のように書きます．

**練習**
ヒープ`h`から最短距離の頂点を取り出し，取り出した頂点を変数`u`に代入するコードになるように，次のセルの`....`の部分に書きなさい．
「最短距離の頂点 0」と頂点1が根に移動したヒープが出力されたら正解．
`extract_min`関数を呼び出すとよい．

In [ ]:
h = demo()
# ヒープhから最短距離の頂点を取り出して変数uに代入するコードを書き加える
....
print('最短距離の頂点', u)
h.draw()

`extract_min`関数の動作について解説する．

1. 最短距離の頂点は木の根にあるので根の頂点を返す．
   根を取り除くので，末尾の頂点を根に移動する．
   
   ![](figs/heap-extract-min-1.svg)
2. 根に移動した頂点はヒープ条件を満たさない可能性がある．
   
   ![](figs/heap-extract-min-2.svg)
4. ヒープ条件を満たすように根の頂点を移動する．
   頂点の移動は`heapify_down`関数で行う．
   
   ![](figs/heap-extract-min-3.svg)

### 3.4 ヒープに頂点を追加する
ヒープ`h`に頂点を追加する関数は`insert`関数です．
頂点の距離は辞書`d`に格納します．

**練習**
頂点0,<u>2,1</u>,3,4,5,6,7をこの順にヒープ`h`に追加するコードになるように，次のセルの`....`の部分を書き換えなさい．
頂点の距離を与えていないので，挿入した順に並んだ二分木が表示されれば正解．

In [ ]:
h = DijkstraHeap()
for u in [0,2,1,3,4,5,6,7]:
  # 頂点uをヒープhに追加するコードを書き加える
  print('追加する頂点', u)
  ....
  h.draw() # ヒープを出力する

ヒープ`h`内の頂点`u`の距離を`ud`にするには`h.d[u]=ud`のように代入する．

**練習**
ヒープに頂点0,<u>2,1</u>,3,4,5,6,7の順に頂点を追加するコードを示す．
頂点の距離を順に0,17,5,20,9,14,29,8にするように`....`の部分を書き換えなさい．
なお，`for`の各繰り返しで，頂点`u`の距離は変数`ud`に格納されている．

In [ ]:
h = DijkstraHeap()
for u, ud in zip([0,2,1,3,4,5,6,7], [0,17,5,20,9,14,29,8]):
  # 頂点uをヒープhに追加するコードを書く
  print('追加する頂点', u, '距離', ud)
  ....
  h.insert(u)
  h.draw() # ヒープを出力する

**練習**

上の練習でできたヒープ`h`がヒープ条件を満たすか否かを次のセルのコメントに答えなさい．
条件を満たさない場合はどの親子の頂点が満たさないか1組の頂点を例示せよ．

In [ ]:
# 答え：このヒープはヒープ条件を満たす・満たさない

### 3.5 新たに追加した頂点についてヒープ条件を満たす

新たに追加した頂点`u`は末尾に追加されます．
この頂点はヒープ条件を満たさない可能性があります．

この頂点についてヒープ条件（どの親も子より距離が短い）を満たすようにヒープを維持するために，この頂点`u`に対して`heapify_up(h.pos[u])`を呼び出します．
なお，`h.pos[u]`は頂点`u`がリスト`Q`の何番目に格納されているかという整数です．

1. 例えば図のように頂点7を追加した直後のヒープは，頂点3と7の親子の間で，ヒープ条件を満たさない可能性がある．
   この例では満たさないが，満たす場合もある．
   
   ![](figs/heap-insert-up-1.svg)
1. 頂点7に対して`heapify_up`関数を呼び出して，ヒープ条件を満たすようにする．
   `h.heapify_up(h.pos[7])`のように呼び出すことで頂点7は適切な位置まで上がる．
   
   ![](figs/heap-insert-up-2.svg)

**練習**
以下のコードは頂点0,2,1, ...を順にヒープ`h`に追加している．
ヒープ条件を満たすように維持したいので，
追加した頂点`u`に対して`heapify_up(h.pos[u])`を呼び出すように次のコードの`....`の部分を書きなさい．

さらに，できたヒープがヒープ条件を満たすか否かを次のセルのコメントに答えなさい．
条件を満たさない場合はどの親子の頂点が満たさないか1組の頂点を例示せよ．

In [ ]:
h = DijkstraHeap()
for u, ud in zip([0,2,1,3,4,5,6,7], [0,17,5,20,9,14,29,8]):
  # 頂点uをヒープhに追加するコードを書く
  print('追加する頂点', u, '距離', ud)
  h.d[u] = ud
  h.insert(u)
  # 追加した頂点uをヒープ条件を満たす位置に移動する
  ....
  h.draw() # ヒープを出力する

# 答え：このヒープはヒープ条件を満たす・満たさない

### 3.6 距離が短くなった頂点についてヒープ条件を満たす

頂点の距離はダイクストラのアルゴリズムの途中で短くなることがあります．
この頂点はヒープ条件を満たさない可能性があります．

この頂点についてヒープ条件（どの親も子より距離が短い）を満たすようにヒープを維持するために，この頂点`u`に対して`heapify_up(h.pos[u])`を呼び出します．

例えば，ヒープ`h`の頂点4の距離を9から7に変更する場合を考えます．
1. 頂点4の距離が短くなったことで，頂点4と親頂点7の間でヒープ条件を満たさない可能性がある．この例では満たさないが，満たす場合もある．
   
   ![](figs/heap-relax-up-1.svg)
2. `heapify_up`関数を呼び出して，ヒープ条件を満たすようにする．
   `h.heapify_up(h.pos[4])`のように呼び出すことで頂点4は適切な位置まで上がる．
   
   ![](figs/heap-relax-up-2.svg)

**練習**
次のコードで頂点4の距離を短くした後にヒープ条件を満たすか否かを次のセルのコメントに答えなさい．
条件を満たさない場合はどの親子の頂点が満たさないか示しなさい．

さらに，`heapify_up`関数を`....`の部分で呼び出して，ヒープ条件を満たすようにしなさい．

In [ ]:
h = demo()
h.d[4] = 7

# 答え：heapify_up関数を呼び出す前の状態はヒープ条件を満たす・満たさない

....
h.draw()

以上です．4章に進みましょう．